# Scrapping Santander ofertas de trabajo

## 1 - Librerías y paquetes

In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sys
import matplotlib.pyplot as plt

## 2 - Obtención de cantidad de páginas con avisos

In [9]:
def getSoup(n):    
    url = f"https://empleo.santander.com/buscar-trabajo/?p={n}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    return soup

In [10]:
soup = getSoup(1)

In [11]:
n = soup.find("span", {"class": "pagination-total-pages"})
paginasTotales = int("".join([x for x in n.string if x.isnumeric()]))

In [12]:
paginasTotales

195

## 3 - Obtención de links de avisos

In [13]:
# Obtiene lista de link de avisos
def getAvisos(soup, avisos):    
    listaItems = soup.find_all("li", {"class":"results-list-item"})
    endpoint = "https://empleo.santander.com/"
    for item in listaItems:
        page = item.a["href"]
        avisos.append(endpoint + page)
    return None

Loop en las páginasTotales para obtener todos los links a avisos.

In [14]:
avisos = []
for x in range(1,paginasTotales+1):
    try:
        soup = getSoup(x)
        getAvisos(soup, avisos)
        if x%5 == 0:
            print(x, end=" ")
    except:
        print(f"\n error on {x}")

5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180 185 190 195 

In [15]:
len(avisos)

2912

## 4 - Extracción de datos de avisos

Extracción de los datos de cabecera de cada aviso.

In [16]:
jobsInfo = []
for n in range(len(avisos)):
    try:
        r = requests.get(avisos[n])
        soup = BeautifulSoup(r.text, "html.parser")
        clases = ["job-location", "job-category", "job-date", "job-id"]
        data = {}
        for clase in clases:
            nodo = soup.find("span", {"class":clase})
            s = list(nodo.strings)
            s = [term.strip(" \n\r") for term in s]
            data[s[0]] = s[1]
        jobsInfo.append(data)
        if n%5==0:
            print(n, end=" ")
    except:
        print(f"error en {n}", end = "\n")

0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180 185 190 195 200 205 210 215 220 225 230 235 240 245 250 255 260 265 270 275 280 285 290 295 300 305 310 315 320 325 330 335 340 345 350 355 360 365 370 375 380 385 390 395 400 405 410 415 420 425 430 435 440 445 450 455 460 465 470 475 480 485 490 495 500 505 510 515 520 525 530 535 540 545 550 555 560 565 570 575 580 585 590 595 600 605 610 615 620 625 630 635 640 645 650 655 660 665 670 675 680 685 690 695 700 705 710 715 720 725 730 735 740 745 750 755 760 765 770 775 780 785 790 795 800 805 810 815 820 825 830 835 840 845 850 855 860 865 870 875 880 885 890 895 900 905 910 915 920 925 930 935 940 945 950 955 960 965 970 975 980 985 990 995 1000 1005 1010 1015 1020 1025 1030 1035 1040 1045 1050 1055 1060 1065 1070 1075 1080 1085 1090 1095 1100 1105 1110 1115 1120 1125 1130 1135 1140 1145 1150 1155 1160 1165 1170 1175 1180 1185 1190 1195 1200 1205 1210 1215 12

## 5 - Carga de los datos al DataFrame

In [40]:
df = pd.DataFrame(jobsInfo)
df

,Lugar,Categoría,PUBLICACIÓN,ID Puesto
0,"São Paulo, Brasil",Product & Service Value Proposition Management...,09/03/2022,Req0934931
1,"Sheffield, Reino Unido",Customer S&S: Remote (NEW),16/02/2022,Req0927008
2,"Milton Keynes, Reino Unido",Customer S&S: Remote (NEW),20/01/2022,Req0915544
3,"Bootle, Reino Unido",Customer S&S: Remote (NEW),07/12/2021,Req0900646
4,"Madrid, España","Project, Product Management",10/03/2022,Req0935337
...,...,...,...,...
2907,"Naucalpan de Juárez, México",Strategy & Corporate Development (NEW),12/02/2020,Req0727970
2908,"Dallas, Estados Unidos",Cumplimiento & Asesoría Jurídica,01/11/2021,okophfw8
2909,"Campeche, México",Customer S&S: F2F (NEW),26/02/2020,Req0729542
2910,"Nueva York, Estados Unidos","Consumer Financing, Corporate & Investment Ban...",11/08/2021,2104001


In [43]:
df["PUBLICACIÓN"] = pd.to_datetime(df["PUBLICACIÓN"],format="%d/%m/%Y")

In [44]:
df

,Lugar,Categoría,PUBLICACIÓN,ID Puesto
0,"São Paulo, Brasil",Product & Service Value Proposition Management...,2022-03-09,Req0934931
1,"Sheffield, Reino Unido",Customer S&S: Remote (NEW),2022-02-16,Req0927008
2,"Milton Keynes, Reino Unido",Customer S&S: Remote (NEW),2022-01-20,Req0915544
3,"Bootle, Reino Unido",Customer S&S: Remote (NEW),2021-12-07,Req0900646
4,"Madrid, España","Project, Product Management",2022-03-10,Req0935337
...,...,...,...,...
2907,"Naucalpan de Juárez, México",Strategy & Corporate Development (NEW),2020-02-12,Req0727970
2908,"Dallas, Estados Unidos",Cumplimiento & Asesoría Jurídica,2021-11-01,okophfw8
2909,"Campeche, México",Customer S&S: F2F (NEW),2020-02-26,Req0729542
2910,"Nueva York, Estados Unidos","Consumer Financing, Corporate & Investment Ban...",2021-08-11,2104001


## 6 - Exportación de los datos

In [45]:
df.to_csv("santanderScrap.csv")